copy data from HDFS folder `/user/pascepet/teplota-usa` to your HDFS-home folder `/user/username`

start HIVE CLI using command

`beeline -u "jdbc:hive2://hador-c1.ics.muni.cz:10000/default;principal=hive/hador-c1.ics.muni.cz@ICS.MUNI.CZ"`

1 Create external table

- Create your database (if not exists)
- Make your database your working database
- Create external table name temperature_tmp

| Column name | Data type |
|:------------|:----------|
| stanice     | string    |
| mesic       | int       |
| den         | int       |
| hodina      | int       |
| teplota     | double    |
| flag        | string    |
| latitude    | double    |
| longitude   | double    |
| vyska       | double    |
| stat        | string    |
| nazev       | string    |

2 Create internal table

- Create internal table named temperature stored as parquet
- Insert data into internal table. Convert temperature data from 10xFahrenheit to celsius using formula $ (\frac{F}{10} - 32) \times \frac{5}{9} $
- Drop external table
- Check that data files are still on HDFS

3 Create internal partitioned table

- Create table partitioned by month use snappy compression
- Insert data into partitioned table
- Inspect partitioned folder on HDFS

To enable dynamic partitioning execute this commands

```
set hive.exec.dynamic.partition=true;
set hive.exec.dynamic.partition.mode=nonstrict;
```


4 Modify creation script into generic load procedure

Use hive vars ?

5 SQL

I. Find state with highest temp in summer


| State | AVG_TEMP |
|:------|:---------|
|       |          |

II. Compare query time for normal and broadcast query with Hash join ?

III. Find states with highest average temperature per month

| Month | State | AVG_TEMP |
|:------|:------|:---------|
|       |       |          |

